#### Code to match candidates naively

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

## Imports

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import torch
torch.cuda.get_device_name(0)

In [ ]:
import os
import ast
import cudf
import glob
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize

pandarallel.initialize(progress_bar=False, use_memory_fs=False)

In [ ]:
from params import *
from utils.metrics import *
from inference.knn import *

## Data

In [ ]:
K = 2

In [ ]:
df = cudf.read_csv(DATA_PATH + "df_train.csv").set_index('id')

In [ ]:
folds = cudf.read_csv(DATA_PATH + f"folds_{K}.csv")[['id', 'fold']]

df = df.merge(folds, how="left", on="id").set_index("id")

In [ ]:
gt_matches = json.load(open(DATA_PATH + "gt.json", 'r'))

In [ ]:
FOLD_IDX = 0
df_val = df[df['fold'] == FOLD_IDX]

### Explo

In [ ]:
# df['country_reverse'] = [c['country_code'] for c in reverse_geocode.search(df[['latitude', 'longitude']].values)]

In [ ]:
# df_diff = df[df['country_reverse'] != df['country']].copy()

In [ ]:
# df_diff['country_reverse_swap'] = [
#     c['country_code'] for c in reverse_geocode.search(df_diff[['longitude', 'latitude']].values)
# ]

# df_diff['country_reverse_swap'] = [
#     c['country_code'] for c in reverse_geocode.search(df_diff[['longitude', 'latitude']].values)
# ]
# df_diff['city_reverse_swap'] = [
#     c['city'] for c in reverse_geocode.search(df_diff[['longitude', 'latitude']].values)
# ]

In [ ]:
# len(df_diff[df_diff['country_reverse_swap'] == df_diff['country']])

In [ ]:
# country = "BR"

In [ ]:
# dfc = df[df['country'] == country].reset_index()


In [ ]:
# import reverse_geocode

# countries = [c['country_code'] for c in reverse_geocode.search(dfc[['latitude', 'longitude']].values)]

In [ ]:
# dfc['country_reverse'] = countries

# dfc_w = dfc[dfc['country_reverse'] != country]

In [ ]:
# dfc_w['country_reverse'].value_counts()

In [ ]:
# len(dfc_w)

# countries_r = [c['country_code'] for c in reverse_geocode.search(dfc_w[['longitude', 'latitude']].values)]

In [ ]:
# plt.figure(figsize=(15, 10))
# plt.scatter(dfc['longitude'], dfc['latitude'], s=1)

# plt.scatter(dfc_w['longitude'], dfc_w['latitude'], s=1)

# # plt.scatter(dfc_w['latitude'], dfc_w['longitude'], s=50)
# # plt.scatter(-dfc_w['longitude'], dfc_w['latitude'], s=50)
# # plt.scatter(dfc_w['longitude'], -dfc_w['latitude'], s=50)

## Position matches

### Base

In [ ]:
# MAX_DIST = None
MAX_DIST = 0.5
NEIGHBORS =  [50, 100]

SAVE = True

In [ ]:
for n_neighbors in NEIGHBORS:
    print(f'\n -> n_neighbors={n_neighbors} - max_dist={MAX_DIST}\n')

    dist_matches = get_nearest_neighbors(df_val, n_neighbors=n_neighbors, max_dist=MAX_DIST)
#     dist_matches = get_nearest_neighbors_country(df_val.to_pandas(), n_neighbors=n_neighbors, max_dist=MAX_DIST)

    found_prop, missed = compute_found_prop(dist_matches, gt_matches)
    n_matches = sum([len(dist_matches[k]) for k in dist_matches])

    print(f'Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')
    print(f'Best reachable IoU : {compute_best_iou(dist_matches, gt_matches) :.3f}')

    if SAVE:
        with open(OUT_PATH + f"dist_matches_{n_neighbors}_{FOLD_IDX}.json", "w") as f:
            json.dump(dist_matches, f)
            print("\n- Saved to ", OUT_PATH + f"dist_matches_{n_neighbors}_{FOLD_IDX}.json")

### +Swapped

In [ ]:
# for max_dist in [None]:
#     for n_neighbors in NEIGHBORS:
#         print(f'\n -> n_neighbors={n_neighbors} - max_dist={max_dist}\n')

#         dist_matches = get_nearest_neighbors_swapped(df_val, n_neighbors=n_neighbors, max_dist=max_dist)

#         found_prop, missed = compute_found_prop(dist_matches, gt_matches)
#         n_matches = sum([len(dist_matches[k]) for k in dist_matches])

#         print(f'Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')
#         print(f'Best reachable IoU : {compute_best_iou(dist_matches, gt_matches) :.3f}')

# #         with open(OUT_PATH + f"dist_matches_{n_neighbors}_{FOLD_IDX}.json", "w") as f:
# #             json.dump(dist_matches, f)
# #             print("\n- Saved to ", OUT_PATH + f"dist_matches_{n_neighbors}_{FOLD_IDX}.json")

# #         break
# #     break

### FPs

In [ ]:
# n_neighbors = 100
# max_dist = 0.5
# lim = 20  # ??

# all_fps = {}
# for fold_idx in tqdm(range(K)):
#     df_val = df[df['fold'] == fold_idx]
#     dist_matches = get_nearest_neighbors(df_val, n_neighbors=n_neighbors)
#     fps = {id_ : " ".join(list(set(dist_matches[id_]) - set(gt_matches[id_]))[:lim]) for id_ in dist_matches}
#     all_fps.update(fps)  # TODO : update ?


In [ ]:
# df_fps = pd.DataFrame.from_dict(all_fps, orient="index").reset_index()
# df_fps.columns = ["id", 'fp_ids']

# triplets = prepare_triplet_data(root=DATA_PATH)
# triplets.drop('fp_ids', axis=1, inplace=True)

# triplets = triplets.merge(df_fps, how="left")
# triplets.to_csv(DATA_PATH + 'triplets_v2.csv', index=False)

## Phone number
- Count encoding for phone numbers present too many times ?
- Phone found twice = match ?

In [ ]:
from inference.knn import find_phone_matches, find_url_matches

In [ ]:
df_val['phone_len'] = df_val[['phone']].to_pandas()['phone'].fillna('').apply(len)

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x=df_val['phone_len'].values.get())
plt.yscale('log')

In [ ]:
df_phone = df_val[df_val['phone_len'] > 5]
df_phone = df_phone[df_phone['phone_len'] < 25]

df_phone = df_phone.to_pandas()

In [ ]:
phone_matches = {}

for country, df_phone_c in tqdm(df_phone.groupby("country")):
#     print(country, len(df_phone_c))
    if country == "US":
        # Group by state
        for state, df_phone_c_s in tqdm(df_phone_c.groupby("state")):
            for id_ in df_phone_c_s.index:
                m = find_phone_matches(id_, df_phone_c_s)
                if len(m):
                    phone_matches[id_] = m
    else:
        for id_ in df_phone_c.index:
            m = find_phone_matches(id_, df_phone_c)
            if len(m):
                phone_matches[id_] = m

In [ ]:
for i in range(1):
    k = list(phone_matches.keys())[i]
    k = np.random.choice(list(phone_matches.keys()))
    display(df.loc[[k] + phone_matches[k]])

In [ ]:
for n_neighbors in NEIGHBORS:
    print(f'\n -> n_neighbors={n_neighbors}\n')

    position_matches = json.load(open(OUT_PATH + f"dist_matches_{n_neighbors}_0.json", 'r'))
    found_prop, missed_pos = compute_found_prop(position_matches, gt_matches)
    n_matches = sum([len(position_matches[k]) for k in position_matches])
    print(f'Position :\t Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')

    merged_matches = {
        k : matches + [m for m in phone_matches.get(k, []) if m not in matches] for k, matches in position_matches.items()
    }
    found_prop, missed = compute_found_prop(merged_matches, gt_matches)
    n_matches = sum([len(merged_matches[k]) for k in merged_matches])
    print(f'Merged :\t found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')

## Url

In [ ]:
min_url_len = 25  # 25

In [ ]:
df_val['url_len'] = df_val[['url']].to_pandas()['url'].fillna('').apply(len)

df_url = df_val[df_val['url_len'] > min_url_len].to_pandas()

len(df_url)

In [ ]:
plt.figure(figsize=(15, 5))
sns.histplot(x=df_url['url_len'], bins=100)
plt.yscale('log')

In [ ]:
url_matches = {}

for country, df_url_c in tqdm(df_url.groupby("country")):
    if country == "US":
        # Group by state
        for state, df_url_c_s in tqdm(df_url_c.groupby("state")):
            for id_ in df_url_c_s.index:
                m = find_url_matches(id_, df_url_c_s)
                if len(m):
                    url_matches[id_] = m
    else:
        for id_ in df_url_c.index:
            m = find_url_matches(id_, df_url_c)
            if len(m):
                url_matches[id_] = m

In [ ]:
for i in range(1):
    k = list(url_matches.keys())[i]
    k = np.random.choice(list(url_matches.keys()))
    display(df.loc[[k] + url_matches[k]])

### Final results & save

In [ ]:
SAVE = True

In [ ]:
for n_neighbors in NEIGHBORS:
    print(f'\n -> n_neighbors={n_neighbors}\n')

    # Pos
    position_matches = json.load(open(OUT_PATH + f"dist_matches_{n_neighbors}_0.json", 'r'))
    found_prop, missed_pos = compute_found_prop(position_matches, gt_matches)
    n_matches = sum([len(position_matches[k]) for k in position_matches])
    print("Position :")
    print(f'Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')
    print(f'Best reachable IoU : {compute_best_iou(position_matches, gt_matches) :.3f}\n')

    # Pos + Phone
    merged_matches = {
        k : matches + [m for m in phone_matches.get(k, []) if m not in matches]
        for k, matches in position_matches.items()
    }
    found_prop, missed = compute_found_prop(merged_matches, gt_matches)
    n_matches = sum([len(merged_matches[k]) for k in merged_matches])
    print("Position + Phone:")
    print(f'Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')
    print(f'Best reachable IoU : {compute_best_iou(merged_matches, gt_matches) :.3f}\n')

    if SAVE:
        with open(OUT_PATH + f"dist-phone_matches_{n_neighbors}_{FOLD_IDX}.json", "w") as f:
            json.dump(merged_matches, f)
#         print("\n- Saved to ", OUT_PATH + f"dist-phone_matches_{n_neighbors}_{FOLD_IDX}.json\n")

    # Pos + Phone + Url
    merged_matches = {
        k : matches + [m for m in url_matches.get(k, []) if m not in matches]
        for k, matches in merged_matches.items()
    }

    found_prop, missed = compute_found_prop(merged_matches, gt_matches)
    n_matches = sum([len(merged_matches[k]) for k in merged_matches])

    print("Position + Phone + Url:")
    print(f'Found {found_prop * 100 :.2f}% of matches with {numerize(n_matches)} candidates.')
    print(f'Best reachable IoU : {compute_best_iou(merged_matches, gt_matches) :.3f}\n')

    if SAVE:
        with open(OUT_PATH + f"dist-phone-url_matches_{n_neighbors}_{FOLD_IDX}.json", "w") as f:
            json.dump(merged_matches, f)
#         print("\n- Saved to ", OUT_PATH + f"dist-phone-url_matches_{n_neighbors}_{FOLD_IDX}.json\n")